In [1]:
# Import librerias 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Rutas de archivos 

# Ruta donde se encuentran los archivos CSV
ruta_datos_mimic = "C:/Users/BiDAlab/TFG_Alfonso/BBDD/mimic-iv-3.1/mimic-iv-3.1/" 

# Importar archivos CSV específicos
admissions = pd.read_csv(ruta_datos_mimic + "hosp/admissions.csv/admissions.csv")
diagnoses = pd.read_csv(ruta_datos_mimic + "hosp/diagnoses_icd.csv/diagnoses_icd.csv")
icu_stays = pd.read_csv(ruta_datos_mimic + "icu/icustays.csv/icustays.csv")

# Ruta guardado
ruta_guardado = "C:/Users/BiDAlab/TFG_Alfonso/Pruebas/Modelo2/MIMIC/CSV/V3/"

In [3]:
# Buscamos en el fichero chartevents (sucesos), los que nos interesan, a través de IDs

# 220050 -> presion sistólica
# 220051 -> presión diastólica
# 220052 -> presión mean ([sistólica + 2*diastólica]/3))

# 223761 -> temp. corporal Fº
# 223762 -> temp. corporal Cº

# 220277 -> saturacion oxigeno (SpO2)
itemids_interes = [220277]  # IDs de ejemplo
chunksize = 1000000

filtered_data = []

for chunk in pd.read_csv(ruta_datos_mimic + "icu/chartevents.csv/chartevents.csv", chunksize=chunksize):
    chunk_filtered = chunk[chunk['itemid'].isin(itemids_interes)]
    filtered_data.append(chunk_filtered)

chartevents = pd.concat(filtered_data)

# Mensaje finalización
print(f"Se ha finalizado la lectura de archivos\n")

Se ha finalizado la lectura de archivos



In [4]:
# Guardamos los datos extraidos de chartevents

chartevents.to_csv(ruta_guardado +"chartevents_oxigeno.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/chartevents_oxigeno.csv'")

Resultados guardados en 'CSV/chartevents_oxigeno.csv'


In [5]:
# Filtramos los pacientes con sepsis con los códigos ICD

# ICD9 : 99591 -> sepsis
#        99592 -> sepsis severa
#        78552 -> septic shock
# ICD10 : A419 -> sepsis no especificada (sangre infectada confirmada pero causa sin especificar)
#         R6521 -> sepsis severa
#         R6520 -> septic shock
codigos_sepsis = ['99591', '99592', '78552', 'R6521', 'A419', 'R6520']

# Filtrar diagnósticos por códigos de sepsis
sepsis_cases = diagnoses[diagnoses['icd_code'].isin(codigos_sepsis)]

# Combinar diagnósticos de sepsis con admisiones para obtener detalles
sepsis_details = pd.merge(sepsis_cases, admissions, on='subject_id', how='inner')

# Mostrar los casos de sepsis
print("Número de casos de sepsis:", sepsis_details['subject_id'].nunique())
print(sepsis_details.head())

Número de casos de sepsis: 15880
   subject_id  hadm_id_x  seq_num icd_code  icd_version  hadm_id_y  \
0    10000826   21086876        2    99591            9   20032235   
1    10000826   21086876        2    99591            9   21086876   
2    10000826   21086876        2    99591            9   28289260   
3    10001401   27012892        3    R6520           10   20144849   
4    10001401   27012892        3    R6520           10   21544441   

             admittime            dischtime deathtime  \
0  2146-12-05 19:07:00  2146-12-12 16:30:00       NaN   
1  2146-12-18 17:39:00  2146-12-24 19:55:00       NaN   
2  2146-12-31 00:43:00  2147-01-02 17:45:00       NaN   
3  2136-11-20 14:20:00  2136-11-23 14:00:00       NaN   
4  2131-06-04 00:00:00  2131-06-15 16:10:00       NaN   

                admission_type admit_provider_id  admission_location  \
0                     EW EMER.            P278S6      EMERGENCY ROOM   
1                 DIRECT EMER.            P61YAW     CLINIC

In [6]:
# Obtenemos una lista con los pacientes unicos con sepsis

n_pacientes_unicos_sepsis = sepsis_details['subject_id'].unique()
print(n_pacientes_unicos_sepsis)
print(f"Número de pacientes únicos: {sepsis_details['subject_id'].nunique()}")

[10000826 10001401 10001843 ... 19995732 19997367 19997886]
Número de pacientes únicos: 15880


In [7]:
# Filtramos chartevents para quedarnos con los sucesos de los casos con ssepsis_cases

chartevents_sepsis = chartevents[chartevents['subject_id'].isin(n_pacientes_unicos_sepsis)]

# Filtrado por pacientes únicos y múltiples itemid
chartevents_sepsis = chartevents[
    (chartevents['subject_id'].isin(n_pacientes_unicos_sepsis))]

# Guardar los detalles de sepsis en un archivo CSV
chartevents_sepsis.to_csv(ruta_guardado +"chartevents_sepsis_oxigeno.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/chartevents_sepsis_oxigeno.csv'")

Resultados guardados en 'CSV/chartevents_sepsis_oxigeno.csv'


In [8]:
# Vamos a añadir como características : 
#       1. Valor max. y min. en las últimas 24 h.
# Trabajaré sobre la última hospitalización del paciente

chartevents_sepsis = pd.read_csv(ruta_guardado + "chartevents_sepsis_oxigeno.csv")

# Borramos columnas inutiles
chartevents_sepsis = chartevents_sepsis.drop(['value', 'valueuom', 'itemid', 'storetime', 'caregiver_id'], axis=1)

# Convertir charttime con formato explícito para cada grupo
chartevents_sepsis['charttime_numeric'] = pd.to_datetime(chartevents_sepsis['charttime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Convertir a segundos desde epoch
chartevents_sepsis['charttime_numeric_s'] = chartevents_sepsis['charttime_numeric'].astype(np.int64) / 10**9
# Obtener la última medición de presión arterial por paciente
ultimo_oxigeno = (
    chartevents_sepsis.groupby('hadm_id')['charttime']
    .max()
    .reset_index()
    .rename(columns={'charttime': 'ultima_charttime'})
)

# Unir con el dataset original para añadir la última hora de medición de cada paciente
df = chartevents_sepsis.merge(ultimo_oxigeno, on='hadm_id', how='left')

df['ultima_charttime'] = pd.to_datetime(df['ultima_charttime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

df['ultima_charttime'] = df['ultima_charttime'].astype(np.int64) / 10**9

df = df.drop(['charttime'],axis= 1)

df.to_csv(ruta_guardado +"borrar.csv", index=False)


In [9]:
print(df.head())

   subject_id   hadm_id   stay_id  valuenum  warning   charttime_numeric  \
0    10001843  26133978  39698942      84.0      0.0 2134-12-06 07:11:00   
1    10001843  26133978  39698942      97.0      0.0 2134-12-06 08:06:00   
2    10001843  26133978  39698942      97.0      0.0 2134-12-06 09:53:00   
3    10001843  26133978  39698942      97.0      0.0 2134-12-06 10:04:00   
4    10001843  26133978  39698942      97.0      0.0 2134-12-05 23:00:00   

   charttime_numeric_s  ultima_charttime  
0         5.204675e+09      5.204686e+09  
1         5.204679e+09      5.204686e+09  
2         5.204685e+09      5.204686e+09  
3         5.204686e+09      5.204686e+09  
4         5.204646e+09      5.204686e+09  


In [11]:
# Paso 1: Identificar la última hospitalización por subject_id
ultima_hospitalizacion = (
    df.groupby('subject_id')['ultima_charttime']
    .max()
    .reset_index()
    .rename(columns={'ultima_charttime': 'max_ultima_charttime'})
)

# Paso 2: Unir esta información con el DataFrame original
df_filtrado = df.merge(ultima_hospitalizacion, on='subject_id', how='left')

# Paso 3: Filtrar solo las filas donde ultima_charttime coincide con la última hospitalización
df_ultima_hosp = df_filtrado[df_filtrado['ultima_charttime'] == df_filtrado['max_ultima_charttime']]

# Paso 4: Opcional: eliminar la columna auxiliar 'max_ultima_charttime' si no es necesaria
df_ultima_hosp = df_ultima_hosp.drop(columns=['max_ultima_charttime', 'stay_id', 'hadm_id'])

# Guardar el resultado final
df_ultima_hosp.to_csv(ruta_guardado + "ultima_hospitalizacion.csv", index=False)

In [21]:
# Filtrar las últimas 24 horas para cada subject_id
def filtrar_ultimas_24h(grupo):
    ultima_hora = grupo['ultima_charttime'].iloc[-1]
    return grupo[grupo['charttime_numeric_s'] >= (ultima_hora - 12*60*60)]

In [22]:
df_filtrado = df_ultima_hosp.groupby('subject_id').apply(filtrar_ultimas_24h).reset_index(drop=True)

# Calcular el valor mínimo y máximo de oxígeno en las últimas 24 horas
resultados = df_filtrado.groupby('subject_id')['valuenum'].agg(
    oxigeno_min_24h='min',
    oxigeno_max_24h='max'
).reset_index()

# Unir los resultados al dataframe original
df_final = df_ultima_hosp.merge(resultados, on='subject_id', how='left')

C:\Users\BiDAlab\AppData\Local\Temp\ipykernel_2988\2965055277.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtrado = df_ultima_hosp.groupby('subject_id').apply(filtrar_ultimas_24h).reset_index(drop=True)


In [23]:
print(df_final.head())

   subject_id  valuenum  warning   charttime_numeric  charttime_numeric_s  \
0    10001843      84.0      0.0 2134-12-06 07:11:00         5.204675e+09   
1    10001843      97.0      0.0 2134-12-06 08:06:00         5.204679e+09   
2    10001843      97.0      0.0 2134-12-06 09:53:00         5.204685e+09   
3    10001843      97.0      0.0 2134-12-06 10:04:00         5.204686e+09   
4    10001843      97.0      0.0 2134-12-05 23:00:00         5.204646e+09   

   ultima_charttime  oxigeno_min_24h  oxigeno_max_24h  
0      5.204686e+09             84.0             99.0  
1      5.204686e+09             84.0             99.0  
2      5.204686e+09             84.0             99.0  
3      5.204686e+09             84.0             99.0  
4      5.204686e+09             84.0             99.0  


In [24]:
# Guardar los detalles de sepsis en un archivo CSV
df_final.to_csv(ruta_guardado +"max_min_oxigeno.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/max_min_oxigeno.csv'")

Resultados guardados en 'CSV/max_min_oxigeno.csv'


In [25]:
# Eliminar duplicados quedándote con el último (keep='last')
ultima_medicion = df_final.drop_duplicates(subset='subject_id', keep='last')

ultima_medicion.to_csv(ruta_guardado +"max_min_oxigeno_unico.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/max_min_oxigeno_unico.csv'")

Resultados guardados en 'CSV/max_min_oxigeno_unico.csv'


In [27]:
# Añado esta característica al fichero de características del MODELO2_V2 (parte ssepsis_cases)

archivo_estadísticas = pd.read_csv("C:/Users/BiDAlab/TFG_Alfonso/Pruebas/Modelo2/MIMIC/CSV/V2/archivo_combiando_oxigeno.csv")

archivo_estadísticas_sepsis = pd.merge(archivo_estadísticas, ultima_medicion, on='subject_id', how='inner')

print(archivo_estadísticas_sepsis.head())

   subject_id  gender  anchor_age  sis_desviacion  dias_desviacion  \
0    10002013       0          53       16.689746         8.576824   
1    10002428       0          80       16.925170        12.605705   
2    10003400       0          72       10.814669         9.766610   
3    10003637       1          57       16.366693         6.206981   
4    10004235       1          47       14.743613        10.639475   

   mean_desviacion   sis_media  dias_media  mean_media  sis_mediana  ...  \
0        13.514515  110.619048   61.476190   76.700000        116.0  ...   
1        15.075749  110.987220   59.006390   79.524845        109.0  ...   
2         9.163913  102.727749   64.884817   78.518325        102.0  ...   
3         7.390996  123.716667   54.166667   71.808333        124.5  ...   
4        11.361041  108.368421   68.131579   79.815789        104.5  ...   

   sepsis  total_warnings_temperatura  total_warnings_oxigeno  valuenum  \
0       1                         0.0          

In [28]:
# Realizamos lo mismo pero para los pacientes sin sepsis

# Ahora hago una lista con los pacientes sin sepsis
subject_ids_sepsis_0 = archivo_estadísticas[archivo_estadísticas['sepsis'] == 0]['subject_id'].tolist()

# Miro en chatevents su oxigeno

# Filtramos chartevents para quedarnos con los sucesos de los casos SIN sepsis
chartevents_no_sepsis = chartevents[chartevents['subject_id'].isin(subject_ids_sepsis_0)]

# Guardar los detalles de sepsis en un archivo CSV
chartevents_no_sepsis.to_csv(ruta_guardado +"chartevents_NO_sepsis_oxigeno.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/chartevents_NO_sepsis_oxigeno.csv'")

Resultados guardados en 'CSV/chartevents_NO_sepsis_oxigeno.csv'


In [29]:
# Vamos a añadir como características : 
#       1. Valor max. y min. en las últimas 24 h.
# Trabajaré sobre la última hospitalización del paciente

chartevents_no_sepsis = pd.read_csv(ruta_guardado + "chartevents_NO_sepsis_oxigeno.csv")

# Borramos columnas inutiles
chartevents_no_sepsis = chartevents_no_sepsis.drop(['value', 'valueuom', 'itemid', 'storetime', 'caregiver_id'], axis=1)

# Convertir charttime con formato explícito para cada grupo
chartevents_no_sepsis['charttime_numeric'] = pd.to_datetime(chartevents_no_sepsis['charttime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Convertir a segundos desde epoch
chartevents_no_sepsis['charttime_numeric_s'] = chartevents_no_sepsis['charttime_numeric'].astype(np.int64) / 10**9
# Obtener la última medición de presión arterial por paciente
ultimo_oxigeno = (
    chartevents_no_sepsis.groupby('hadm_id')['charttime']
    .max()
    .reset_index()
    .rename(columns={'charttime': 'ultima_charttime'})
)

# Unir con el dataset original para añadir la última hora de medición de cada paciente
df = chartevents_no_sepsis.merge(ultimo_oxigeno, on='hadm_id', how='left')

df['ultima_charttime'] = pd.to_datetime(df['ultima_charttime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

df['ultima_charttime'] = df['ultima_charttime'].astype(np.int64) / 10**9

df = df.drop(['charttime'],axis= 1)

df.to_csv(ruta_guardado +"borrar_no_sepsis.csv", index=False)


In [30]:
# Paso 1: Identificar la última hospitalización por subject_id
ultima_hospitalizacion = (
    df.groupby('subject_id')['ultima_charttime']
    .max()
    .reset_index()
    .rename(columns={'ultima_charttime': 'max_ultima_charttime'})
)

# Paso 2: Unir esta información con el DataFrame original
df_filtrado = df.merge(ultima_hospitalizacion, on='subject_id', how='left')

# Paso 3: Filtrar solo las filas donde ultima_charttime coincide con la última hospitalización
df_ultima_hosp = df_filtrado[df_filtrado['ultima_charttime'] == df_filtrado['max_ultima_charttime']]

# Paso 4: Opcional: eliminar la columna auxiliar 'max_ultima_charttime' si no es necesaria
df_ultima_hosp = df_ultima_hosp.drop(columns=['max_ultima_charttime', 'stay_id', 'hadm_id'])

# Guardar el resultado final
df_ultima_hosp.to_csv(ruta_guardado + "ultima_hospitalizacion_NO_sepsis.csv", index=False)

In [31]:
df_filtrado_NO_sepsis = df_ultima_hosp.groupby('subject_id').apply(filtrar_ultimas_24h).reset_index(drop=True)

# Calcular el valor mínimo y máximo de oxígeno en las últimas 24 horas
resultados = df_filtrado_NO_sepsis.groupby('subject_id')['valuenum'].agg(
    oxigeno_min_24h='min',
    oxigeno_max_24h='max'
).reset_index()

# Unir los resultados al dataframe original
df_final = df_ultima_hosp.merge(resultados, on='subject_id', how='left')

C:\Users\BiDAlab\AppData\Local\Temp\ipykernel_2988\3652146005.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtrado_NO_sepsis = df_ultima_hosp.groupby('subject_id').apply(filtrar_ultimas_24h).reset_index(drop=True)


In [32]:
# Guardar los detalles de sepsis en un archivo CSV
df_final.to_csv(ruta_guardado +"max_min_oxigeno_NO_sepsis.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/max_min_oxigeno_NO_sepsis.csv'")

Resultados guardados en 'CSV/max_min_oxigeno_NO_sepsis.csv'


In [33]:
# Eliminar duplicados quedándote con el último (keep='last')
ultima_medicion = df_final.drop_duplicates(subset='subject_id', keep='last')

ultima_medicion.to_csv(ruta_guardado +"max_min_oxigeno_unico_NO_sepsis.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/max_min_oxigeno_unico_NO_sepsis.csv'")

Resultados guardados en 'CSV/max_min_oxigeno_unico_NO_sepsis.csv'


In [34]:
# Añado esta característica al fichero de características del MODELO2_V2 (parte ssepsis_cases)

archivo_estadísticas_NO_sepsis = pd.merge(archivo_estadísticas, ultima_medicion, on='subject_id', how='inner')

print(archivo_estadísticas_NO_sepsis.head())

   subject_id  gender  anchor_age  sis_desviacion  dias_desviacion  \
0    10002155       0          80        9.555103         7.056912   
1    10002760       1          56        9.953648         4.998814   
2    10004606       0          64       24.549849         7.013110   
3    10004764       1          70       13.304104         4.511060   
4    10005866       1          57       13.605328         5.313636   

   mean_desviacion   sis_media  dias_media  mean_media  sis_mediana  ...  \
0         8.105554  113.400000   57.400000   80.200000        115.0  ...   
1         5.984499  108.434783   57.521739   74.217391        107.0  ...   
2        12.932371  152.151515   51.939394   80.606061        154.0  ...   
3         5.075356  118.581395   45.162791   67.651163        120.0  ...   
4        12.339189  116.017241   57.103448   75.086207        114.5  ...   

   sepsis  total_warnings_temperatura  total_warnings_oxigeno  valuenum  \
0       0                         0.0          

In [41]:
# Combino los archivos de sepsis y de no ssepsis_cases

archivo_final = pd.concat([archivo_estadísticas_sepsis, archivo_estadísticas_NO_sepsis])

archivo_final = archivo_final.drop(columns=['warning', 'charttime_numeric', 'charttime_numeric_s', 'ultima_charttime', 'valuenum'])

# Ordenar por subject_id de menor a mayor
archivo_final = archivo_final.sort_values(by='subject_id').reset_index(drop=True)

# Guardar los detalles en un archivo CSV
archivo_final.to_csv(ruta_guardado +"archivo_combiando_oxigeno_max_min.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/archivo_combiando_oxigeno_max_min.csv'")

Resultados guardados en 'CSV/archivo_combiando_oxigeno_max_min.csv'
